# L2: Evaluate Inputs: Classification

## Setup
#### Load the API key and relevant Python libaries.
In this course, we've provided some code that loads the OpenAI API key for you.

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [ ]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

For tasks in which lots of independent sets 
of instructions are needed to handle different cases, 
it can be beneficial to first **classify the type of query**, 
and then use that classification to **determine which instructions to use**. 

This can be achieved by defining fixed categories and hard-coding 
instructions that are relevant for handling tasks in 
a given category.

For instance, when building a customer service assistant, 
it might be important to first classify the type of query, 
and then determine which instructions to use based on 
that classification. 

#### Classify customer queries to handle different cases

A **DELIMITER** is just a way to separate different parts of 
an instruction or output, and it helps the model kind 
of determine the different sections. And so for this example, we'll 
use the delimiter 4 hashtags. And this is a nice 
delimiter because it's actually <u>represented as one token.</u>

 

In [ ]:
delimiter = "####"
system_message = f"""
You will be provided with customer service queries. \
The customer service query will be delimited with \
{delimiter} characters.
Classify each query into a primary category \
and a secondary category. 
Provide your output in json format with the \
keys: primary and secondary.

Primary categories: Billing, Technical Support, \
Account Management, or General Inquiry.

Billing secondary categories:
Unsubscribe or upgrade
Add a payment method
Explanation for charge
Dispute a charge

Technical Support secondary categories:
General troubleshooting
Device compatibility
Software updates

Account Management secondary categories:
Password reset
Update personal information
Close account
Account security

General Inquiry secondary categories:
Product information
Pricing
Feedback
Speak to a human

"""
user_message = f"""\
I want you to delete my profile and all of my user data"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] # we'll just format this into a messages list with the system message and the user message delimited with these hashtags. 
response = get_completion_from_messages(messages)
print(response)

*OUTPUT*
```json
{
  "primary": "Account Management",
  "secondary": "Close account"
}
```

The nice thing about asking for a 
structured output like a JSON is that you can then read this 
easily into some kind of object, so a dictionary for example in Python, 
or something else if you're using a different language, and 
then you can use this as the input to a subsequent step. 

In [ ]:
user_message = f"""\
Tell me more about your flat screen tvs"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

*OUTPUT*
```json
{
  "primary": "General Inquiry",
  "secondary": "Product information"
}
```

In general, based on the categorization 
of the customer inquiry, we can now provide a set of more specific 
instructions to handle the next steps.